## Introduction to Multi-Agent Framework
A multi-agent framework refers to a system where multiple autonomous agents collaborate, interact, and work together to accomplish tasks that would otherwise be difficult or time-consuming for a single agent. Each agent typically has a specific role or goal and brings its own expertise to the system.

## Key Components:

### Agents:

Independent entities within the system that perform tasks.
Collaboration: Agents working together to achieve a common goal.

### Memory:

The ability of agents to remember past interactions and utilize that information in future tasks.

### Focus:

The specialization of agents, each focusing on a specific sub-task to break down complex problems.

### Reflection:

Agents' ability to review their actions, learn, and improve.

### Guardrails:

Mechanisms to ensure that agents follow a set of rules and avoid looping or incorrect behaviors.

# Key Elements of Multi-Agent Framework

## 1.  Role Playing

Each agent has a distinct role, which allows it to specialize in a certain part of the task. Role playing is the foundation of the multi-agent system, as it ensures that each agent contributes effectively to the overall goal.

In our example, the agents have the following roles:

- Text Extraction Agent: Extracts text from the provided PDF contract.
- Comparison Agent: Compares the extracted data with predefined reference data.
- Reporting Agent: Generates an email-friendly report summarizing the analysis.

## 2. Focus

Focus refers to how agents specialize in particular sub-tasks, each of them focusing on specific elements of the broader task. By dividing a large task into smaller, focused sub-tasks, agents can perform them efficiently.

For example, in the contract analysis:

- The Text Extraction Agent focuses on extracting text from the contract.
- The Comparison Agent focuses on comparing the extracted data to the reference standards.
- The Reporting Agent focuses on generating a report summarizing the analysis.

## 3. Tools

Each agent uses tools to complete its task. A tool could be a function, an API, or any external resource that assists an agent in performing its task.

In our case, agents use the following tools:

- PDF Text Extractor: For extracting text from the PDF contract.
- Reference Data Comparator: For comparing the extracted data to predefined reference standards.
- Email Report Generator: For creating a detailed report summarizing the analysis.

## 4. Cooperation

Cooperation in multi-agent systems is when agents collaborate, sharing information and working together to accomplish the final goal. Agents rely on each other's output to proceed with their task. This collaboration is key to the success of the overall system.

In the contract analysis example:

- The Text Extraction Agent extracts text, which is then passed to the Comparison Agent.
- The Comparison Agent checks the extracted data against the reference data, and the results are passed to the Reporting Agent.
- The Reporting Agent uses the comparison results to generate a final report.

## 5. Guardrails
Guardrails are mechanisms that ensure agents behave correctly and avoid issues such as infinite loops or repeating tasks unnecessarily. Guardrails enforce limits and constraints to ensure the system remains efficient and accurate.

For example, we can use guardrails to:

- Ensure that each task is executed only once (avoiding redundant processing).
- Make sure that tasks follow the right order (e.g., text extraction before comparison).
- Prevent agents from getting stuck in loops (e.g., if the text extraction fails, it should not be retried endlessly).

## 6. Memory
Memory is the ability of agents to store and retrieve information. This allows them to keep track of previous interactions and use that information for future tasks. In multi-agent systems, memory can be divided into three types:

- Short-term memory: Temporary data stored for a single task.
- Long-term memory: Persistent data that can be shared across tasks.
- Entity memory: Memory related to specific entities (like documents or companies).

Each agent in our system uses memory to store outputs of its tasks and share that information with other agents.

In [ ]:
!pip install PyPDF2

In [ ]:
pip install --upgrade crewai crewai-tools

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os

# add you openai key
openai_api_key = ''
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [ ]:
from crewai import Crew, Process, Agent, Task
from crewai.tools import BaseTool
import re
import json
import PyPDF2

# This class creates custom tool that can take a PDF converts it to text
class PDFReaderTool(BaseTool):
    name: str = "pdf_text_extractor"
    description: str = "this tool reads a pdf and convert it to text for further analysis i.e key-term extraction"

    def _run(self, pdf_file: str) -> str:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text


# Define agents
text_extraction_agent = Agent(
    role="Text Extraction Specialist",
    goal="Extract precise key information from {pdf_file}",
    backstory="An accomplished paralegal with unmatched expertise in parsing complex documents.",
    verbose=True,
    allow_delegation=False,
    tools=[PDFReaderTool()]
)

comparison_agent = Agent(
    role="Document Comparison Expert",
    goal="Compare extracted document data against reference standards i.e {reference_data}",
    backstory="A meticulous auditor with an eagle eye for discrepancies, ensuring all information aligns perfectly.",
    verbose=True,
    allow_delegation=False
)

reporting_agent = Agent(
    role="Detailed Report Generator",
    goal="Create professional and actionable analysis reports",
    backstory="A seasoned communicator adept at transforming technical data into clear, insightful reports.",
    verbose=True,
    allow_delegation=False
)

# Define reference data
reference_data = {
    "limitation of liability": "30 days",
    "owner expiry date": "30 days",
    "notice period": "30 days",
    "agreement duration": "12 months",
    "payment terms": "30 days",
    "confidentiality period": "3 years",
    "insurance coverage": "$1,000,000",
    "maximum monthly hours": "120"
}

# Perform text extraction task
extraction_task = Task(
    description="""
    Carefully extract key information from the following document text.
    Focus on finding these specific items: {reference_keys}
    """,
    agent=text_extraction_agent,
    expected_output="A detailed dictionary of extracted key-value pairs from the document"
)

# Comparison task
comparison_task = Task(
    description="""
    Compare the extracted data against these reference standards:
    {reference_data_json}
    Identify any mismatches, missing information, or potential discrepancies.
    Provide a detailed explanation for each difference found.
    """,
    agent=comparison_agent,
    expected_output="A comprehensive list of mismatches and detailed analysis"
)

# Reporting task
reporting_task = Task(
    description="""
    Generate a comprehensive report summarizing the document analysis.
    Include:
    1. Extracted key information
    2. Comparison results
    3. Detailed insights and recommendations
    """,
    agent=reporting_agent,
    expected_output="A detailed, structured report with all analysis findings"
)

# Combine everything in Crew
crew = Crew(
    agents=[
        text_extraction_agent,
        comparison_agent,
        reporting_agent
    ],
    tasks=[
        extraction_task,
        comparison_task,
        reporting_task
    ],
    memory=True,
    verbose=True,
    process=Process.sequential
)

# Inputs
inputs = {
    "pdf_file": 'sample_contract.pdf', # change this your specific contract
    "reference_data": reference_data,
    "reference_keys": ", ".join(reference_data.keys()),  # Inject the reference keys into the description
    "reference_data_json": json.dumps(reference_data, indent=2)  # Convert reference data to JSON format
}

# Execute tasks
result = crew.kickoff(inputs=inputs)

# Print result for verification
print(result)


# Agent: Text Extraction Specialist
## Task: 
    Carefully extract key information from the following document text.
    Focus on finding these specific items: limitation of liability, owner expiry date, notice period, agreement duration, payment terms, confidentiality period, insurance coverage, maximum monthly hours
    


# Agent: Text Extraction Specialist
## Thought: I should use the pdf_text_extractor tool to extract text from the sample_contract.pdf document and then carefully search for the specific items mentioned in the task.
## Using tool: pdf_text_extractor
## Tool Input: 
"{\"pdf_file\": \"sample_contract.pdf\"}"
## Tool Output: 
CONSULTING SERVICES AGREEMENT  
 
This Consulting Services Agreement ("Agreement") is made effective as of 15/01/2024, by and between:  
 
ABC Technologies Inc.  
123 Tech Park Avenue  
Silicon Valley, CA 94025  
 
Contact: john.doe@abctech.com  
 
and 
 
XYZ Consulting LLC ("Consultant")  
456 Business Plaza  
New York, NY 10001  
 
KEY TERMS AN